In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.read_csv('/content/drive/MyDrive/24-1 인공지능/080228-master/실습13_자료/ratings.csv')
print( df1.shape )
print( df1.head(2) )

df2 = pd.read_csv('/content/drive/MyDrive/24-1 인공지능/080228-master/실습13_자료/movies.csv')
print( df2.shape )
print( df2.head(2) )

(100004, 4)
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
(9125, 3)
   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1        2    Jumanji (1995)                   Adventure|Children|Fantasy


In [2]:
df4 = pd.merge(df1, df2, on='movieId')
print( df4.shape )
print( df4.head(2) )

(100004, 6)
   userId  movieId  rating   timestamp                   title genres
0       1       31     2.5  1260759144  Dangerous Minds (1995)  Drama
1       7       31     3.0   851868750  Dangerous Minds (1995)  Drama


In [8]:
# 사용자 기반 협업 필터링
movie_user_rating = df4.pivot_table('rating', index='userId', columns='title')
print( movie_user_rating.shape )
print( movie_user_rating.head(3) )

(671, 9064)
title   "Great Performances" Cats (1998)  $9.99 (2008)  \
userId                                                   
1                                    NaN           NaN   
2                                    NaN           NaN   
3                                    NaN           NaN   

title   'Hellboy': The Seeds of Creation (2004)  \
userId                                            
1                                           NaN   
2                                           NaN   
3                                           NaN   

title   'Neath the Arizona Skies (1934)  'Round Midnight (1986)  \
userId                                                            
1                                   NaN                     NaN   
2                                   NaN                     NaN   
3                                   NaN                     NaN   

title   'Salem's Lot (2004)  'Til There Was You (1997)  'burbs, The (1989)  \
userId                     

In [10]:
# NaN 값을 0으로 대체
new_rating = movie_user_rating.fillna(0)
print( new_rating.head(3) )

title   "Great Performances" Cats (1998)  $9.99 (2008)  \
userId                                                   
1                                    0.0           0.0   
2                                    0.0           0.0   
3                                    0.0           0.0   

title   'Hellboy': The Seeds of Creation (2004)  \
userId                                            
1                                           0.0   
2                                           0.0   
3                                           0.0   

title   'Neath the Arizona Skies (1934)  'Round Midnight (1986)  \
userId                                                            
1                                   0.0                     0.0   
2                                   0.0                     0.0   
3                                   0.0                     0.0   

title   'Salem's Lot (2004)  'Til There Was You (1997)  'burbs, The (1989)  \
userId                                 

In [11]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

item_collabor = cosine_similarity( new_rating )
print( item_collabor )

[[1.         0.         0.         ... 0.06291708 0.         0.01746565]
 [0.         1.         0.12429498 ... 0.02413984 0.17059464 0.1131753 ]
 [0.         0.12429498 1.         ... 0.08098382 0.13660585 0.17019275]
 ...
 [0.06291708 0.02413984 0.08098382 ... 1.         0.04260878 0.08520194]
 [0.         0.17059464 0.13660585 ... 0.04260878 1.         0.22867673]
 [0.01746565 0.1131753  0.17019275 ... 0.08520194 0.22867673 1.        ]]


In [12]:
# 사용자별 유사성 정보 값 확인
similarity_matrix = pd.DataFrame(data=item_collabor, index=new_rating.index, columns=new_rating.index)
print( similarity_matrix.head() )

userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.000000  0.000000  0.074482  0.016818  0.000000  0.083884   
2       0.000000  1.000000  0.124295  0.118821  0.103646  0.000000  0.212985   
3       0.000000  0.124295  1.000000  0.081640  0.151531  0.060691  0.154714   
4       0.074482  0.118821  0.081640  1.000000  0.130649  0.079648  0.319745   
5       0.016818  0.103646  0.151531  0.130649  1.000000  0.063796  0.095888   

userId       8         9         10   ...       662       663       664  \
userId                                ...                                 
1       0.000000  0.012843  0.000000  ...  0.000000  0.000000  0.014481   
2       0.113190  0.113333  0.043213  ...  0.477306  0.063202  0.077784   
3       0.249781  0.134475  0.114672  ...  0.161205  0.064198  0.176222   
4       0.191013  0.030417  0.137186  ...  0.114319  0.047228  0

In [15]:
# 추천 기능 구현
# userId = 3인 사용자에 대해서, 유사도 상위 5명의 사용자 출력
def get_item_collabor(userId):
    return similarity_matrix[userId].sort_values(ascending=False)[:6]

print( get_item_collabor(3) )

userId
3      1.000000
379    0.319619
146    0.295441
513    0.281170
94     0.262043
458    0.259552
Name: 3, dtype: float64
